In [ ]:
import numpy as np
import glob
import imageio.v2 as imageio
import vedo
vedo.settings.default_backend= 'vtk'
from vedo import Plane
import brainrender
from brainrender import Scene
from brainglobe_atlasapi import BrainGlobeAtlas
import matplotlib.pyplot as plt
import re
from pathlib import Path


In [ ]:
import random
from pathlib import Path
import itertools

import numpy as np
from myterial import orange
from rich import print

from brainrender import Scene
from brainrender.actors import Points
from matplotlib import colors as mcolors
from tqdm import tqdm
import pandas as pd
data_path = Path("C:/Microscope_images/processed/becalia_rabies_barseq/BRAC8498.3e/ara_spots")
ara_barcode_spots = pd.read_pickle(data_path / "ara_barcode_spots.pkl")
ara_starters = pd.read_pickle(data_path / "ara_starter_cells.pkl")


def read_in_BrainJ_cells(cells, regions, acronyms, not_in=None):
    if len(regions) > 0:
        cells = cells[cells['area_id'].isin(regions)]
    if len(acronyms) > 0:
        cells = cells[cells['area_acronym'].isin(acronyms)]
    if not_in is not None:
        cells = cells[~cells['area_acronym'].isin(not_in)]
    Z = cells['ara_z'].tolist()
    Y = cells['ara_y'].tolist()
    X = cells['ara_x'].tolist()
    
    AtlasRes = 1000
    X = [element * AtlasRes for element in X]
    Y = [element * AtlasRes for element in Y]
    Z = [element * AtlasRes for element in Z]
    
    if len(X) == 0:
        return np.empty((0, 3))
    
    pts = [[x, y, z] for x, y, z in zip(X, Y, Z)]
    return np.vstack(pts)

def get_cycled_colors(n):
    colors = list(mcolors.CSS4_COLORS.keys())
    random.shuffle(colors)
    return [colors[i % len(colors)] for i in range(n)]

# Get unique barcodes present in both DataFrames
common_barcodes = set(ara_barcode_spots['barcode'].unique()) & set(ara_starters['main_barcode'].unique())

# Filter DataFrames to only include common barcodes
ara_barcode_spots = ara_barcode_spots[ara_barcode_spots['barcode'].isin(common_barcodes)]
ara_starters = ara_starters[ara_starters['main_barcode'].isin(common_barcodes)]

# Function to render specific barcodes
def render_selected_barcodes(selected_barcodes=None):
    if selected_barcodes is None:
        # Get all barcodes
        selected_barcodes = common_barcodes
    elif type(selected_barcodes) == int:
        selected_barcodes = set(itertools.islice(common_barcodes, selected_barcodes))
    unique_barcodes = list(set(selected_barcodes) & common_barcodes)
    colors = get_cycled_colors(len(unique_barcodes))
    barcode_to_color = dict(zip(unique_barcodes, colors))

    # Group cells by barcode and color
    for barcode in tqdm(unique_barcodes, desc="Processing barcodes", unit="barcode"):
        # Get points for current barcode
        rabies_cells = read_in_BrainJ_cells(
            ara_starters[(ara_starters["starter"]==False) & (ara_starters["main_barcode"]==barcode)],
            [],
            [],
            ["outside"]
        )
        
        starter_cells = read_in_BrainJ_cells(
            ara_starters[(ara_starters["starter"]==True) & (ara_starters["main_barcode"]==barcode)],
            [],
            [],
            ["outside"]
        )
        
        rabies_spots = read_in_BrainJ_cells(
            ara_barcode_spots[ara_barcode_spots["barcode"]==barcode],
            [],
            [],
            ["outside"]
        )

        color = barcode_to_color[barcode]

        if rabies_cells.size > 0:
            scene.add(Points(
                rabies_cells,
                name=f"rabies_cells_{barcode}",
                colors=color,
                alpha=0.2,
                radius=10
            ))
        if starter_cells.size > 0:
            scene.add(Points(
                starter_cells,
                name=f"starter_cells_{barcode}",
                colors=color,
                alpha=0.4,
                radius=20
            ))
        if rabies_spots.size > 0:
            scene.add(Points(
                rabies_spots,
                name=f"rabies_spots_{barcode}",
                colors=color,
                alpha=1,
                radius=2
            ))

In [ ]:
import numpy as np
import matplotlib.colors as mcolors


def generate_shades(base_color, num_shades):
    base_rgb = mcolors.to_rgb(base_color)
    # Adjust the range of alpha values to create a more extreme gradient
    return [mcolors.to_hex((base_rgb[0] * alpha, base_rgb[1] * alpha, base_rgb[2] * alpha)) for alpha in np.linspace(1, 0.1, num_shades)]

# Define the broad areas and their base colors
large_actors = {
    "AUDpo" : "brown",
    "VISp" : "blue",
    "VISpl" : "lightgreen",
    "VISl" : "mediumspringgreen",
    "VISal" : "lime",
    "VISpm" : "deepskyblue",
    "VISli" : "cyan",
    "RSP" : "deeppink",
    "TEa" : "gold",
    'TH' : "blueviolet",
    "AUDd" : "yellow",
    "AUDv" : "sandybrown",
    "AUDp" : "orange",
}

# Sub-areas that need to be shaded
infected_areas = {   
    ### Auditory primary
    'AUDp1': "AUDp",
    'AUDp2/3': "AUDp",
    'AUDp4': "AUDp",
    'AUDp5': "AUDp",
    'AUDp6a': "AUDp",
    'AUDp6b': "AUDp",
    ### Auditory posterior
    'AUDpo1': "AUDpo",
    'AUDpo2/3': "AUDpo",
    'AUDpo4': "AUDpo",
    'AUDpo5': "AUDpo",
    'AUDpo6a': "AUDpo",
    'AUDpo6b': "AUDpo",
    ### Auditory ventral
    'AUDv2/3': "AUDv",
    'AUDv4': "AUDv",
    'AUDv5': "AUDv",
    'AUDv6a': "AUDv",
    'AUDv6b': "AUDv",
    ### Thalamus
    'IGL': "TH",
    "LGd-ip" : "TH",
    "LGd-sh" : "TH",
    'LGd-co': "TH",
    'MG': "TH",
    'MGd': "TH",
    'MGv': "TH",
    "LAT" : "TH",
    'IntG': "TH",
    'LGd': "TH",
    'LGv': "TH",
    "VENT" : "TH",
    "PP" : "TH",
    "PIL" : "TH",
    "VPM" : "TH",
    "VPMpc" : "TH",
    "VM" : "TH",
    "POL" : "TH",
    ### Retrosplenial lateral agranular
    'RSPagl1': "RSP",
    'RSPagl2/3': "RSP",
    'RSPagl5': "RSP",
    'RSPagl6a': "RSP",
    'RSPagl6b': "RSP",
    ### Retrosplenial dorsal
    'RSPd1': "RSP",
    'RSPd2/3': "RSP",
    'RSPd5': "RSP",
    'RSPd6a': "RSP",
    'RSPd6b': "RSP",
    ### Retrosplenial ventral
    'RSPv1': "RSP",
    'RSPv2/3': "RSP",
    'RSPv5': "RSP",
    'RSPv6a': "RSP",
    ### Visual antero-lateral
    'VISal1': "VISal",
    'VISal2/3': "VISal",
    'VISal4': "VISal",
    'VISal5': "VISal",
    'VISal6a': "VISal",
    'VISal6b': "VISal",
    ### Visual lateral
    'VISl1': "VISl",
    'VISl2/3': "VISl",
    'VISl4': "VISl",
    'VISl5': "VISl",
    'VISl6a': "VISl",
    'VISl6b': "VISl",
    ### Visual laterointermediate
    'VISli1': "VISli",
    'VISli2/3': "VISli",
    'VISli4': "VISli",
    'VISli5': "VISli",
    'VISli6a': "VISli",
    'VISli6b': "VISli",
    ### Visual primary
    'VISp1': "VISp",
    'VISp2/3': "VISp",
    'VISp4': "VISp",
    'VISp5': "VISp",
    'VISp6a': "VISp",
    'VISp6b': "VISp",
    ### Visual posteromedial
    'VISpm1': "VISpm",
    'VISpm2/3': "VISpm",
    'VISpm4': "VISpm",
    'VISpm5': "VISpm",
    'VISpm6a': "VISpm",
    'VISpm6b': "VISpm",
    ### TEa
    'TEa1': "TEa",
    'TEa2/3': "TEa",
    'TEa4': "TEa",
    'TEa5': "TEa",
    'TEa6a': "TEa",
    'TEa6b': "TEa",
}

# Create the gradient colors for the infected areas
infected_colors = {}
for base_area, base_color in large_actors.items():
    num_shades = len([key for key in infected_areas if infected_areas[key] == base_area])
    shades = generate_shades(base_color, num_shades)
    for area in [key for key in infected_areas if infected_areas[key] == base_area]:
        infected_colors[area] = shades.pop(0)

#import ace_tools as tools; tools.display_dataframe_to_user(name="Infected Areas Colors", dataframe=pd.DataFrame.from_dict(infected_colors, orient='index', columns=['Color']))

infected_colors



In [ ]:
unimportant_areas = [
    'APN',
    ###
    'CA1',
    'CA3',
    'DG-mo',
    'DG-po',
    'DG-sg',
    ### Ectorhinal
    'ECT5',
    'ECT6a',
    ### Entorhinal
    'ENTl5',
    ###
    'LP',
    'MB',
    'MGd',
    'MGv',
    'MRN',
    'ND',
    'PAG',
    ###
    'PERI6a',
    'PERI6b',
    ###
    'POL',
    ###
    'PoT',
    ###
    'SCig',
    'SCop',
    'SCsg',
    'SCzo',
    'SGN',
    ### Temporal association
    'TEa1',
    'TEa2/3',
    'TEa4',
    'TEa5',
    'TEa6a',
    'TEa6b',
    ###
    'alv',
    'ar',
    'bsc',
    'cing',
    'dhc',
    'ec',
    'fiber tracts',
    'fp',
    'or',
    'scwm'
]

In [ ]:
import brainrender
# Initialize brainrender scene
brainrender.settings.BACKGROUND_COLOR = [
    0.10,
    0.10,
    0.10,
]  # change rendering background color
brainrender.settings.WHOLE_SCREEN = (
    True  # make the rendering window be smaller
)
brainrender.settings.OFFSCREEN = False
brainrender.settings.SHOW_AXES = False # turn off the axes display
brainrender.settings.ROOT_ALPHA= (0.1)
brainrender.settings.ROOT_COLOR = "white"
brainrender.settings.DEFAULT_ATLAS = "allen_mouse_10um"
brainrender.settings.SHADER_STYLE = "cartoon"
brainrender.settings.LW = 0

tab20 = plt.colormaps["tab20"]



# Create a brainrender scene with the custom plotter
#scene = Scene(plotter_class=CustomPlotter)
scene = Scene()

large_actors = {
    "AUDpo" : "brown",
    "VISp" : "blue",
    "VISpl" : "lightgreen",
    "VISl" : "mediumspringgreen",
    "VISal" : "lime",
    "VISpm" : "deepskyblue",
    "VISli" : "cyan",
    "RSP" : "deeppink",
    "TEa" : "gold",
    'TH' : "blueviolet",
    "AUDd" : "yellow",
    "AUDv" : "sandybrown",
    "AUDp" : "orange",
}

large_non_cortical_actors = [
    "HPF",
    "ENT",
    "PERI",
    "ECT",
    "BS",
    "fiber tracts",
    "OLF",
    "CTXsp"
]
all_actors = ara_barcode_spots["area_acronym"].unique().astype(str)
all_actors = all_actors.tolist()
all_actors.remove("root")
brainreg_positions1 = [
    np.array([8621.842 , 397.942 , 5706.5796], dtype="float32"),
    np.array([9575.921 , 214.3228, 5678.3687], dtype="float32")
]
normal_vectors1 = [(-1.8919415, -0.12186934, 0.03463937), (1.8919415, 0.12186943, -0.03463934)]

atlas = BrainGlobeAtlas("allen_mouse_10um", check_latest=False)

# Add infected areas
infected_actors_list = []
for i, actor in enumerate(infected_colors.items()):
    #print(actor[0])
    #print(added_actor._mesh)
    added_actor = scene.add_brain_region(actor[0], alpha=0.07, color= actor[1], hemisphere="left", silhouette=False)
    #scene.add_silhouette(added_actor, lw=0.00001, color=actor[1])


    infected_actors_list.append(added_actor)

large_actors_list = []
for i, actor in enumerate(large_actors.items()):
    added_actor = scene.add_brain_region(actor[0], alpha=0.001, color= actor[1])# hemisphere="left")
    #print(actor[0])
    #print(added_actor._mesh)
    scene.add_silhouette(added_actor, lw=1, color=actor[1])
    #scene.add_label(added_actor, actor,color= tab20(i % 20)[:3])
    large_actors_list.append(added_actor)

large_non_cortical_actors_list = []
for i, actor in enumerate(large_non_cortical_actors):
    added_actor = scene.add_brain_region(actor, alpha=0.05, color= "grey",)# hemisphere="left")
    #print(actor)
    #print(added_actor._mesh)
    scene.add_silhouette(added_actor, lw=0.0001, color="grey")
    #scene.add_label(added_actor, actor,color= tab20(i % 20)[:3])
    large_non_cortical_actors_list.append(added_actor)

selected_barcodes = ["AGACTATGCTAAGC", "TTAATACGGGCTTT", "TACCATTAAGGCTG"]  # Replace with actual barcodes
render_selected_barcodes(20) #selected_barcodes)
plane = Plane(pos=brainreg_positions1[0], s=(10000,5000), normal=normal_vectors1[1]) 
plane2 = Plane(pos=brainreg_positions1[1], s=(10000,5000), normal=normal_vectors1[0])
scene.slice(plane, actors=infected_actors_list+large_actors_list+large_non_cortical_actors_list, close_actors=True)
scene.slice(plane2, actors=infected_actors_list+large_actors_list+large_non_cortical_actors_list, close_actors=True)


# Initialize a list to store camera information
camera_info = []

# Create a function to print the camera position, focal point, distance, and zoom
def print_camera_info(event):
    cam = scene.plotter.camera
    cam_pos = cam.GetPosition()
    focal_point = cam.GetFocalPoint()
    distance = cam.GetDistance()
    zoom = scene.plotter.camera.GetParallelScale() if scene.plotter.camera.GetParallelProjection() else scene.plotter.camera.GetViewAngle()
    view_up = cam.GetViewUp()
    info = {
        "position_x": cam_pos[0],
        "position_y": cam_pos[1],
        "position_z": cam_pos[2],
        "focal_point_x": focal_point[0],
        "focal_point_y": focal_point[1],
        "focal_point_z": focal_point[2],
        "distance": distance,
        "zoom": zoom,
        "view_up_x": view_up[0],
        "view_up_y": view_up[1],
        "view_up_z": view_up[2]
    }
    
    # Store the information in the list
    camera_info.append(info)
    
    # Print all information in one line
    print(f"Position: {cam_pos}, Focal Point: {focal_point}, Distance: {distance}, Zoom: {zoom}")

# Add a callback to the plotter to print the camera information
scene.plotter.add_callback('RightButtonPress', print_camera_info)

custom_camera={
    "pos": (4000.692883, -1007.944676, -8000.266488),
    "viewup": ( 0.334511,  -0.941771,     0.0342),
    "clipping_range": (31983, 76783),
    "focal_point": (15858.069514,    4644.458311,	-7535.302423),
    "distance": 36264.153611
    }
root_actor = scene.get_actors('root')[0]
root_actor.opacity(0)
# Render the scene with the interactive GUI
scene.render(camera=custom_camera, interactive=True) #


# Convert the list of camera information to a DataFrame
camera_df = pd.DataFrame(camera_info)

# Display the DataFrame
print("Stored Camera Information DataFrame:")
print(camera_df)

In [ ]:
good_camera_df = camera_df.copy()

# Good renderer

In [ ]:
import brainrender
import numpy as np
import pandas as pd
from brainrender import Scene, Animation
from vedo import Plane
import matplotlib.pyplot as plt
import itertools

brainrender.settings.BACKGROUND_COLOR = [0.10, 0.10, 0.10]
brainrender.settings.WHOLE_SCREEN = True
brainrender.settings.OFFSCREEN = True
brainrender.settings.SHOW_AXES = False
brainrender.settings.ROOT_ALPHA = 0.1
brainrender.settings.ROOT_COLOR = "white"
brainrender.settings.DEFAULT_ATLAS = "allen_mouse_25um"
brainrender.settings.SHADER_STYLE = "cartoon"
brainrender.settings.LW = 0

tab20 = plt.colormaps["tab20"]

# Create a brainrender scene
scene = Scene()

large_actors = {
    "AUDpo": "brown",
    "VISp": "blue",
    "VISpl": "lightgreen",
    "VISl": "mediumspringgreen",
    "VISal": "lime",
    "VISpm": "deepskyblue",
    "VISli": "cyan",
    "RSP": "deeppink",
    "TEa": "gold",
    'TH': "blueviolet",
    "AUDd": "yellow",
    "AUDv": "sandybrown",
    "AUDp": "orange",
}

large_non_cortical_actors = [
    "HPF",
    "ENT",
    "PERI",
    "ECT",
    "BS",
    "fiber tracts",
    "OLF",
    "CTXsp"
]

all_actors = ["root"]  # Dummy data to avoid undefined variable error
brainreg_positions1 = [
    np.array([8621.842, 397.942, 5706.5796], dtype="float32"),
    np.array([9575.921, 214.3228, 5678.3687], dtype="float32")
]
normal_vectors1 = [(-1.8919415, -0.12186934, 0.03463937), (1.8919415, 0.12186943, -0.03463934)]

atlas = BrainGlobeAtlas("allen_mouse_10um", check_latest=False)


# Initialize a list to store camera information
camera_info = []

# Create a function to print the camera position, focal point, distance, and zoom
def print_camera_info(event):
    cam = scene.plotter.camera
    cam_pos = cam.GetPosition()
    focal_point = cam.GetFocalPoint()
    distance = cam.GetDistance()
    zoom = scene.plotter.camera.GetParallelScale() if scene.plotter.camera.GetParallelProjection() else scene.plotter.camera.GetViewAngle()
    
    info = {
        "position_x": cam_pos[0],
        "position_y": cam_pos[1],
        "position_z": cam_pos[2],
        "focal_point_x": focal_point[0],
        "focal_point_y": focal_point[1],
        "focal_point_z": focal_point[2],
        "distance": distance,
        "zoom": zoom
    }
    
    # Store the information in the list
    camera_info.append(info)
    
    # Print all information in one line
    print(f"Position: {cam_pos}, Focal Point: {focal_point}, Distance: {distance}, Zoom: {zoom}")

# Add a callback to the plotter to print the camera information
scene.plotter.add_callback('keypress', print_camera_info)

# Create video maker
vmaker = Animation(scene, Path.cwd(), "brainrender_video_highqual")

# Define callback function to show sliced large actors and large non-cortical actors
def show_sliced_large_actors(scene, frame, total_frames):
    # Add large actors
    large_actors_list = []
    for i, actor in enumerate(large_actors.items()):
        added_actor = scene.add_brain_region(actor[0], alpha=0.001, color=actor[1])
        scene.add_silhouette(added_actor, lw=1, color=actor[1])
        large_actors_list.append(added_actor)

    # Add large non-cortical actors
    large_non_cortical_actors_list = []
    for i, actor in enumerate(large_non_cortical_actors):
        added_actor = scene.add_brain_region(actor, alpha=0.05, color="grey")
        scene.add_silhouette(added_actor, lw=0.0001, color="grey")
        large_non_cortical_actors_list.append(added_actor)
    plane = Plane(pos=brainreg_positions1[0], s=(10000, 5000), normal=normal_vectors1[1])
    plane2 = Plane(pos=brainreg_positions1[1], s=(10000, 5000), normal=normal_vectors1[0])
    scene.slice(plane, actors=large_actors_list + large_non_cortical_actors_list, close_actors=True)
    scene.slice(plane2, actors=large_actors_list + large_non_cortical_actors_list, close_actors=True)

# Define callback function to show sliced infected actors
def show_sliced_infected_actors(scene, frame, total_frames):
    print("Showing sliced infected actors")
    infected_actors_list = []
    for i, actor in enumerate(infected_colors.items()):
        #print(actor[0])
        added_actor = scene.add_brain_region(actor[0], alpha=0.07, color=actor[1], hemisphere="left", silhouette=False)
        infected_actors_list.append(added_actor)
    plane = Plane(pos=brainreg_positions1[0], s=(10000, 5000), normal=normal_vectors1[1])
    plane2 = Plane(pos=brainreg_positions1[1], s=(10000, 5000), normal=normal_vectors1[0])
    scene.slice(plane, actors=infected_actors_list, close_actors=True)
    scene.slice(plane2, actors=infected_actors_list, close_actors=True)

# Define the render_selected_barcodes function without tqdm
def render_selected_barcodes(selected_barcodes=None):
    if selected_barcodes is None:
        # Get all barcodes
        selected_barcodes = common_barcodes
    elif type(selected_barcodes) == int:
        selected_barcodes = set(itertools.islice(common_barcodes, selected_barcodes))
    unique_barcodes = list(set(selected_barcodes) & set(common_barcodes))
    colors = get_cycled_colors(len(unique_barcodes))
    barcode_to_color = dict(zip(unique_barcodes, colors))

    # Group cells by barcode and color
    for barcode in unique_barcodes:  # Removed tqdm for debugging
        # Get points for current barcode
        rabies_cells = read_in_BrainJ_cells(
            ara_starters[(ara_starters["starter"] == False) & (ara_starters["main_barcode"] == barcode)],
            [],
            [],
            ["outside"]
        )
        
        starter_cells = read_in_BrainJ_cells(
            ara_starters[(ara_starters["starter"] == True) & (ara_starters["main_barcode"] == barcode)],
            [],
            [],
            ["outside"]
        )
        
        rabies_spots = read_in_BrainJ_cells(
            ara_barcode_spots[ara_barcode_spots["barcode"] == barcode],
            [],
            [],
            ["outside"]
        )

        color = barcode_to_color[barcode]

        if rabies_cells.size > 0:
            scene.add(Points(
                rabies_cells,
                name=f"rabies_cells_{barcode}",
                colors=color,
                alpha=0.2,
                radius=10
            ))
        if starter_cells.size > 0:
            scene.add(Points(
                starter_cells,
                name=f"starter_cells_{barcode}",
                colors=color,
                alpha=0.4,
                radius=20
            ))
        if rabies_spots.size > 0:
            scene.add(Points(
                rabies_spots,
                name=f"rabies_spots_{barcode}",
                colors=color,
                alpha=1,
                radius=2
            ))

# Prepare common barcodes and data for rendering
data_path = Path("C:/Microscope_images/processed/becalia_rabies_barseq/BRAC8498.3e/ara_spots")
ara_barcode_spots = pd.read_pickle(data_path / "ara_barcode_spots.pkl")
ara_starters = pd.read_pickle(data_path / "ara_starter_cells.pkl")

# Define read_in_BrainJ_cells and get_cycled_colors functions
def read_in_BrainJ_cells(cells, regions, acronyms, not_in=None):
    if len(regions) > 0:
        cells = cells[cells['area_id'].isin(regions)]
    if len(acronyms) > 0:
        cells = cells[cells['area_acronym'].isin(acronyms)]
    if not_in is not None:
        cells = cells[~cells['area_acronym'].isin(not_in)]
    Z = cells['ara_z'].tolist()
    Y = cells['ara_y'].tolist()
    X = cells['ara_x'].tolist()
    
    AtlasRes = 1000
    X = [element * AtlasRes for element in X]
    Y = [element * AtlasRes for element in Y]
    Z = [element * AtlasRes for element in Z]
    
    if len(X) == 0:
        return np.empty((0, 3))
    
    pts = [[x, y, z] for x, y, z in zip(X, Y, Z)]
    return np.vstack(pts)

def get_cycled_colors(n):
    colors = list(mcolors.CSS4_COLORS.keys())
    random.shuffle(colors)
    return [colors[i % len(colors)] for i in range(n)]

# Get unique barcodes present in both DataFrames
common_barcodes = list(set(ara_barcode_spots['barcode'].unique()) & set(ara_starters['main_barcode'].unique()))
#test_limit = 50
#common_barcodes = common_barcodes[:test_limit]

# Filter DataFrames to only include common barcodes
ara_barcode_spots = ara_barcode_spots[ara_barcode_spots['barcode'].isin(common_barcodes)]
ara_starters = ara_starters[ara_starters['main_barcode'].isin(common_barcodes)]


# Keyframe 0: Start zoomed out from an angle
vmaker.add_keyframe(
    0,
    camera={
    "pos": (-163510.45445820506, -211818.18624394896, 107045.79069077772),
    "viewup": (0, -1, 0),
    "clipping_range": (31983, 76783),
    "focal_point": (7829.739799214466, 4296.026746533125, -5694.49796940642),
    "distance": 297948.29738034646
    },
    zoom=1
)

# Keyframe 1: Move in closer to the brain
vmaker.add_keyframe(
    10,
    camera={
    "pos": (-65155.224930,	-73343.886028,	37210.135701),
    "viewup": (0, -1, 0),
    "clipping_range": (31983, 76783),
    "focal_point": (7829.739799,	4296.026747,	-5694.497969),
    "distance": 114871.966652
    },
    zoom=1
)

# Keyframe 2: Move to a frontal position
vmaker.add_keyframe(
    20,
    camera={
    "pos": (-20956.499012,	1598.608394,	-5645.557678),
    "viewup": (0, -1, 0),
    "clipping_range": (31983, 76783),
    "focal_point": (7834.890271,	4165.833307,	-5636.436126),
    "distance": 14842.230469
    },
    zoom=1
)

# Keyframe 3: Hold that position
vmaker.add_keyframe(
    22,
    camera={
    "pos": (-20956.499012,	1598.608394,	-5645.557678),
    "viewup": (0, -1, 0),
    "clipping_range": (31983, 76783),
    "focal_point": (7834.890271,	4165.833307,	-5636.436126),
    "distance": 14842.230469
    },
    zoom=1
)

# Keyframe 3: Show sliced large actors and large non-cortical actors
vmaker.add_keyframe(
    23,
    camera={
    "pos": (-20956.499012,	1598.608394,	-5645.557678),
    "viewup": (0, -1, 0),
    "clipping_range": (31983, 76783),
    "focal_point": (7834.890271,	4165.833307,	-5636.436126),
    "distance": 14842.230469
    },
    zoom=1,
    callback=show_sliced_large_actors
    )

# Keyframe 4: Show sliced infected actors
vmaker.add_keyframe(
    30,
    camera={
    "pos": (-20956.499012,	1598.608394,	-5645.557678),
    "viewup": (0, -1, 0),
    "clipping_range": (31983, 76783),
    "focal_point": (7834.890271,	4165.833307,	-5636.436126),
    "distance": 14842.230469
    },
    zoom=1,
    callback=show_sliced_infected_actors
    )

# Keyframe 3: Hold that position
vmaker.add_keyframe(
    60,
    camera={
    "pos": (-20956.499012,	1598.608394,	-5645.557678),
    "viewup": (0, -1, 0),
    "clipping_range": (31983, 76783),
    "focal_point": (7834.890271,	4165.833307,	-5636.436126),
    "distance": 14842.230469
    },
    zoom=1
)

# Keyframe 3: Zoom out from frontal
vmaker.add_keyframe(
    70,
    camera={
    "pos": (-87026.182778,	407.159312,	-5649.790989),
    "viewup": (0, -1, 0),
    "clipping_range": (31983, 76783),
    "focal_point": (7834.890271,	4165.833307,	-5636.43612),
    "distance": 64935.509630
    },
    zoom=1
)

# Keyframe 4: Move to a upper position
vmaker.add_keyframe(
    75,
    camera={
    "pos": (14195.096893212758, -109809.87667053597, -3203.3762570010786),
    "viewup": (0.997618,   0.068551,  -0.007656),
    "clipping_range": (31983, 76783),
    "focal_point": (6305.223994013311, 4770.690515059968, -5350.991573444866),
    "distance": 114871.96665194607
    },
    zoom=1
)


# Keyframe 4: Zoom into upper position
vmaker.add_keyframe(
    80,
    camera={
    "pos": (14195.096893212758, -30809.87667053597, -3203.3762570010786),
    "viewup": (0.997618,   0.068551,  -0.007656),
    "clipping_range": (31983, 76783),
    "focal_point": (6305.223994013311, 4770.690515059968, -5350.991573444866),
    "distance": 114871.96665194607
    },
    zoom=1
)

# Keyframe 4: Hold upper position
vmaker.add_keyframe(
    140,
    camera={
    "pos": (14195.096893212758, -30809.87667053597, -3203.3762570010786),
    "viewup": (0.997618,   0.068551,  -0.007656),
    "clipping_range": (31983, 76783),
    "focal_point": (6305.223994013311, 4770.690515059968, -5350.991573444866),
    "distance": 114871.96665194607
    },
    zoom=1
)

# Keyframe 5: Move to a frontal position
vmaker.add_keyframe(
    145,
    camera={
    "pos": (-38006.226065,	-6802.751214,	-4756.497002),
    "viewup": (0, -1, 0),
    "clipping_range": (31983, 76783),
    "focal_point": (6302.300431,	4066.227106,	-5476.308856),
    "distance": 94935.509630
    },
    zoom=1
)

# Define a callback function to remove the root brain mesh from the scene
def remove_root_brain(scene, frame, total_frames):
    root_actor = scene.get_actors('root')[0]
    scene.remove(root_actor)
    print("Root brain mesh removed from the scene")

# Keyframe 6: Remove root
vmaker.add_keyframe(
    150,
    camera={
    "pos": (-38006.226065,	-6802.751214,	-4756.497002),
    "viewup": (0, -1, 0),
    "clipping_range": (31983, 76783),
    "focal_point": (6302.300431,	4066.227106,	-5476.308856),
    "distance": 94935.509630
    },
    zoom=1,
    callback=remove_root_brain
)


# Keyframe 5: Zoom into infected area
vmaker.add_keyframe(
    160,
    camera={
    "pos": (-2553.692883, -2047.944676, -10000.266488),
    "viewup": ( 0.334511,  -0.941771,     0.0342),
    "clipping_range": (31983, 76783),
    "focal_point": (15858.069514,    4644.458311,	-7535.302423),
    "distance": 36264.153611
    },
    zoom=1
)


# Define the callback function to add barcodes incrementally
def add_barcodes_incrementally(scene, frame, total_frames):
    global added_barcodes
    # Calculate the number of barcodes to add based on the frame
    if len(added_barcodes) < 30:
        num_barcodes = 1  # First 10 barcodes: 1 per second
    elif len(added_barcodes) < 250:
        num_barcodes = 10  # Next 100 barcodes: 10 per second
    else:
        num_barcodes = 100  # Remaining barcodes: 100 per second
    
    # Determine the new barcodes to add
    new_barcodes = list(itertools.islice(common_barcodes, len(added_barcodes), len(added_barcodes) + num_barcodes))
    added_barcodes.extend(new_barcodes)
    render_selected_barcodes(new_barcodes)
    print(f"Added barcodes: {new_barcodes}")

# Add keyframes for adding barcodes incrementally
added_barcodes = []
# Define the number of barcodes for each stage
first_stage_barcodes = 30
second_stage_barcodes = 250
remaining_barcodes = len(common_barcodes) - (first_stage_barcodes + second_stage_barcodes)

# Calculate the number of keyframes required for each stage
first_stage_keyframes = first_stage_barcodes  # 1 barcode per keyframe
second_stage_keyframes = second_stage_barcodes // 10  # 10 barcodes per keyframe
remaining_stage_keyframes = (remaining_barcodes + 99) // 100  # 100 barcodes per keyframe, ceiling division

# Total number of keyframes
total_keyframes = 170 + first_stage_keyframes + second_stage_keyframes + remaining_stage_keyframes


for frame in range(170, total_keyframes):
    vmaker.add_keyframe(
        frame,
        camera={
            "pos": (-2553.692883, -2047.944676, -10000.266488),
            "viewup": ( 0.334511,  -0.941771,     0.0342),
            "clipping_range": (31983, 76783),
            "focal_point": (15858.069514,    4644.458311,	-7535.302423),
            "distance": 36264.153611
        },
        zoom=1,
        callback=add_barcodes_incrementally
        )


# Keyframe 5: Zoom into infected area
vmaker.add_keyframe(
    total_keyframes + 10,
    camera={
    "pos": (4000.692883, -1007.944676, -8000.266488),
    "viewup": ( 0.334511,  -0.941771,     0.0342),
    "clipping_range": (31983, 76783),
    "focal_point": (15858.069514,    4644.458311,	-7535.302423),
    "distance": 36264.153611
    },
    zoom=1
)
total_keyframes += 10

# Keyframe 5: pan infected area
vmaker.add_keyframe(
    total_keyframes + 10,
    camera={
    "pos": (760.9075903071061, -6800.78538540021, -16825.953826529476),
    "viewup": ( 0.629375,  -0.766265,   0.129328),
    "clipping_range": (31983, 76783),
    "focal_point": (25241.50034339267, 17832.986967211807, 9993.803901083016),
    "distance": 43879.62586930941
    },
    zoom=1
)
total_keyframes += 10

# Keyframe 5: pan infected area
vmaker.add_keyframe(
    total_keyframes + 10,
    camera={
    "pos": (4478.584446588466, -9169.56688087612, -29822.18279305565),
    "viewup": ( 0.395164 , -0.850584 ,  0.346917),
    "clipping_range": (31983, 76783),
    "focal_point": (13951.842397583103, 14273.251159486239, 16865.089364689422),
    "distance":  53094.347301864254
    },
    zoom=1
)
total_keyframes += 10

# Keyframe 5: pan infected area
vmaker.add_keyframe(
    total_keyframes + 10,
    camera={
    "pos": (20845.138794959097, -6154.10581368232, -28893.475186677686),
    "viewup": ( -0.053359 , -0.945052 ,  0.322537 ),
    "clipping_range": (31983, 76783),
    "focal_point": (-5167.558467034369, 10104.710745190683, 14442.424257913592),
    "distance":  53094.347301863796
    },
    zoom=1
)
total_keyframes += 10

# Keyframe 5: pan infected area
vmaker.add_keyframe(
    total_keyframes + 10,
    camera={
    "pos": (30474.199492520165, -4568.039851240409, -21612.45498009715),
    "viewup": (-0.204645 , -0.972840 ,  0.108180  ),
    "clipping_range": (31983, 76783),
    "focal_point": (-13659.587309022345, 7701.088152535213, 5233.160602225766),
    "distance":  53094.347301863796
    },
    zoom=1
)
total_keyframes += 10

# Calculate the duration based on the total number of keyframes
fps = 1 # Frame rate
duration = total_keyframes  # Duration of the video
print(f"Total keyframes: {total_keyframes}, Duration: {duration}")
render_kwargs = {
    "width": 1920,
    "height": 1080,
}
vmaker.make_video(fps=1, duration=duration, render_kwargs=render_kwargs)


In [ ]:
# Keyframe 3: Show sliced large actors and large non-cortical actors
vmaker.add_keyframe(2, camera="frontal", zoom=1, callback=show_sliced_large_actors)

# Keyframe 4: Show sliced infected actors
vmaker.add_keyframe(3, camera="frontal", zoom=1, callback=show_sliced_infected_actors)

# Keyframe 5: Move to a top camera view
vmaker.add_keyframe(4, camera="top", zoom=1)

# Keyframe 7: Move to a frontal view
vmaker.add_keyframe(6, camera="frontal", zoom=1)

# Add keyframes for adding barcodes incrementally
added_barcodes = []
# Define the number of barcodes for each stage
first_stage_barcodes = 10
second_stage_barcodes = 100
remaining_barcodes = len(common_barcodes) - (first_stage_barcodes + second_stage_barcodes)

# Calculate the number of keyframes required for each stage
first_stage_keyframes = first_stage_barcodes  # 1 barcode per keyframe
second_stage_keyframes = second_stage_barcodes // 10  # 10 barcodes per keyframe
remaining_stage_keyframes = (remaining_barcodes + 99) // 100  # 100 barcodes per keyframe, ceiling division

# Total number of keyframes
total_keyframes = 70 + first_stage_keyframes + second_stage_keyframes + remaining_stage_keyframes

# Calculate the duration based on the total number of keyframes
fps = 10  # Frame rate
duration = total_keyframes / fps  # Duration of the video
print(f"Total keyframes: {total_keyframes}, Duration: {duration}")

for frame in range(7, total_keyframes):
    vmaker.add_keyframe(frame, camera="frontal", zoom=1, callback=add_barcodes_incrementally)

total_keyframes = 30
# Calculate the duration based on the limited number of barcodes
fps = 10  # Frame rate
duration = total_keyframes + 7 / fps  # Duration of the video

In [ ]:
# Correcting the data manually to fit the columns
corrected_data = [
    [0, -163510.454458, -211818.186244, 107045.790691, 7829.739799, 4296.026747, -5694.497969, 297948.297381, 7.5],
    [1, -163510.454458, -211818.186244, 107045.790691, 7829.739799, 4296.026747, -5694.497969, 297948.297381, 7.5],
    [2, -65155.224930, -73343.886028, 37210.135701, 7829.739799, 4296.026747, -5694.497969, 114871.966652, 7.5],
    [3, -65155.224930, -73343.886028, 37210.135701, 7829.739799, 4296.026747, -5694.497969, 114871.966652, 7.5],
    [4, -56956.499012, 1598.608394, -5645.557678, 7834.890271, 4165.833307, -5636.436126, 64842.230469, 7.5],
    [5, -56956.499012, 1598.608394, -5645.557678, 7834.890271, 4165.833307, -5636.436126, 64842.230469, 7.5],
    [6, -87026.182778, 407.159312, -5649.790989, 7834.890271, 4165.833307, -5636.436126, 94935.509630, 7.5],
    [7, -87026.182778, 407.159312, -5649.790989, 7834.890271, 4165.833307, -5636.436126, 94935.509630, 7.5],
    [8, 15029.545571, -110462.637346, -3881.591874, 6302.300431, 4066.227106, -5476.308856, 114871.966652, 7.5],
    [9, 15029.545571, -110462.637346, -3881.591874, 6302.300431, 4066.227106, -5476.308856, 114871.966652, 7.5],
    [10, -88006.226065, -6802.751214, -4756.497002, 6302.300431, 4066.227106, -5476.308856, 94935.509630, 7.5],
    [11, -88006.226065, -6802.751214, -4756.497002, 6302.300431, 4066.227106, -5476.308856, 94935.509630, 7.5],
    [12, -29346.838229, -4781.464885, -10919.744495, 6508.258583, 2141.097121, -8434.151348, 36601.748666, 7.5],
    [13, -29346.838229, -4781.464885, -10919.744495, 6508.258583, 2141.097121, -8434.151348, 36601.748666, 7.5]
]

# Define the columns
columns = ['index', 'position_x', 'position_y', 'position_z', 'focal_point_x', 'focal_point_y', 'focal_point_z', 'distance', 'zoom']
import pandas as pd
# Create the dataframe
df_corrected = pd.DataFrame(corrected_data, columns=columns)

# Format float columns to 6 decimal places
formatted_df_corrected = df_corrected.applymap(lambda x: f"{x:.6f}" if isinstance(x, float) else x)

# Show the formatted dataframe
formatted_df_corrected

In [ ]:
import brainrender
import numpy as np
from brainrender import Scene, Animation
from vedo import Plane

brainrender.settings.BACKGROUND_COLOR = [0.12, 0.12, 0.12]
brainrender.settings.WHOLE_SCREEN = False
brainrender.settings.SHOW_AXES = False
brainrender.settings.ROOT_ALPHA = 0.5
brainrender.settings.ROOT_COLOR = "white"
brainrender.settings.DEFAULT_ATLAS = "allen_mouse_25um"
brainrender.settings.SHADER_STYLE = "cartoon"
brainrender.settings.LW = 0
brainrender.settings.OFFSCREEN = True

# Create a brainrender scene
scene = Scene()

# Adding brain regions as described
large_actors = {
    "AUDpo": "brown",
    "VISp": "blue",
    "VISpl": "lightgreen",
    "VISl": "mediumspringgreen",
    "VISal": "lime",
    "VISpm": "deepskyblue",
    "VISli": "cyan",
    "RSP": "deeppink",
    "TEa": "gold",
    "TH": "blueviolet",
    "AUDd": "yellow",
    "AUDv": "sandybrown",
    "AUDp": "orange",
}

large_non_cortical_actors = [
    "HPF",
    "ENT",
    "PERI",
    "ECT",
    "BS",
    "fiber tracts",
    "OLF",
    "CTXsp"
]

# Initially, do not add actors to the scene
added_actors = []

brainreg_positions1 = [
    np.array([8621.842, 397.942, 5706.5796], dtype="float32"),
    np.array([9575.921, 214.3228, 5678.3687], dtype="float32")
]
normal_vectors1 = [(-1.8919415, -0.12186934, 0.03463937), (1.8919415, 0.12186943, -0.03463934)]

plane = Plane(pos=brainreg_positions1[0], s=(10000, 5000), normal=normal_vectors1[1])
plane2 = Plane(pos=brainreg_positions1[1], s=(10000, 5000), normal=normal_vectors1[0])

# Create video maker
vmaker = Animation(scene, "folder", "brainrender_video")

# Define callback function to show sliced actors
def show_sliced_actors(scene, frame, total_frames):
    global added_actors
    for actor in large_actors.keys():
        added_actor = scene.add_brain_region(actor, alpha=0.1, color=large_actors[actor])
        added_actors.append(added_actor)
    for actor in large_non_cortical_actors:
        added_actor = scene.add_brain_region(actor, alpha=0.1, color="grey")
        added_actors.append(added_actor)
    scene.slice(plane, actors=added_actors)
    scene.slice(plane2, actors=added_actors, close_actors=True)

# Define callback function to hide actors
def hide_actors(scene, frame, total_frames):
    global added_actors
    for actor in added_actors:
        scene.remove(actor)
    added_actors = []

# Keyframe 1: Initial state with only root visible
vmaker.add_keyframe(0, camera="frontal", zoom=1)

# Keyframe 2: Show sliced brain regions using callback
vmaker.add_keyframe(1, camera="frontal", zoom=1, callback=show_sliced_actors)

# Keyframe 3: Pan to a top view
vmaker.add_keyframe(2, camera="top", zoom=1)

# Keyframe 4: Pan to a sagittal view
vmaker.add_keyframe(3, camera="sagittal", zoom=1)

# Keyframe 5: Hide the actors
vmaker.add_keyframe(4, camera="frontal", zoom=1, callback=hide_actors)

# Render the video
vmaker.make_video(fps=10)


In [ ]:
# Initialize brainrender scene
scene = Scene()
tab20 = plt.colormaps["tab20"]

folder_path = 'C:/Microscope_data/BRAC8498.3e/overviews/'
# Create a list of all .tif files ending with .ome.tif_Coords.tif
tif_files = glob.glob(f'{folder_path}/*.ome.tif_Coords.tif')
#Order the files alphabetically
tif_files.sort()
# Select the first and last file
first_slice = tif_files[0]
last_slice = tif_files[-1]

rotate = False  

# Iterate over the list of files
for i, file_path in enumerate(tif_files):

    # Load the image
    img = imageio.imread(file_path)  # Adjust this path

    # img.shape is (3, height, width)
    # Let's pick three points: top-left corner, top-right corner, and bottom-left corner of the image
    p1 = np.array([img[0, 0, 0], img[1, 0, 0], img[2, 0, 0]])  # Top-left corner
    p2 = np.array([img[0, 0, -1], img[1, 0, -1], img[2, 0, -1]])  # Top-right corner
    p3 = np.array([img[0, -1, 0], img[1, -1, 0], img[2, -1, 0]])  # Bottom-left corner

    #Let's find the central point of the image too
    p4 = np.array(
                [img[0, img.shape[1] // 2, img.shape[2] // 2],
                 img[1, img.shape[1] // 2, img.shape[2] // 2],
                 img[2, img.shape[1] // 2, img.shape[2] // 2]])

    # Compute vectors
    v1 = p2 - p1
    v2 = p3 - p1

    # Compute the normal vector to the plane defined by v1 and v2
    normal_vector = np.cross(v1, v2)
    normal_vector = normal_vector / np.linalg.norm(normal_vector)  # Normalize the vector
    #normal_vector = (normal_vector[2], normal_vector[1], normal_vector[0])  # Adjust the order of the elements
    brainreg_position = (p4[0] * 1000, p4[1] * 1000, p4[2] * 1000)

    c = tab20(i % 20)[:3]

    # Add the plane to the scene
    plane = Plane(pos=brainreg_position, s=(10000, 5000), normal=tuple(normal_vector), c=c) #  
    scene.add(plane)

# Render the scene
scene.render()


In [ ]:
# 0 1 2
# straight down midline
# 0 2 1
# straight down midline but flipped direction
# 1 0 2
# flat across midline (brown up)
# 1 2 0
# correct plane, blue first
# 2 0 1
#crash
# 2 1 0
#correct plane, rotated 90 degrees, blue first


In [ ]:
import numpy as np
import imageio
from brainglobe_atlasapi import BrainGlobeAtlas

folder_path = 'C:/Microscope_data/BRAC8498.3e/overviews/'
# Create a list of all .tif files ending with .ome.tif_Coords.tif
tif_files = glob.glob(f'{folder_path}/*.ome.tif_Coords.tif')

# Iterate over the list of files
for i, file_path in enumerate(tif_files):
    test_path = file_path

def get_unique_actors_from_tif(file_path):
    """ Given a path to a .tif file, this function returns a list of unique actors in the image.

    Args:
        file_path (str): Path to the .tif file

    Returns:
        list: List of unique actors in the image
    """
    # Load the BrainGlobe Atlas
    bg_atlas = BrainGlobeAtlas("allen_mouse_25um", check_latest=False)
    
    # Load the image
    img = imageio.imread(file_path)
    
    # Assume img.shape is (3, height, width)
    height, width = img.shape[1], img.shape[2]
    
    # Calculate step sizes for the grid
    step_x = 1
    step_y = 1
    
    # Initialize an empty set for unique actors
    unique_actors = set()
    
    # Sample points in a 100x100 grid
    for i in range(0, height, step_y):
        for j in range(0, width, step_x):
            # Convert image coordinates to atlas coordinates
            # Assume that the conversion to atlas coordinates is similar to brainreg_position calculation
            x, y = j, i
            coord = (img[2, i, j] * 1000, img[1, i, j] * 1000, img[0, i, j] * 1000)
            # Query the atlas for the structure at this coordinate
            try:
                structure = bg_atlas.structure_from_coords(coord, microns=True, as_acronym=True,  hierarchy_lev=2)
            except:
                structure = None
            # Add the result to the set of unique actors
            if structure is not None:
                unique_actors.add(structure)
    
    # Convert the set to a list to return
    return list(unique_actors)
